In [1]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import expm
import scipy
import scipy.sparse
from matplotlib import animation, rc, pyplot as plt

In [2]:
# define harmonic oscillator per normal mode
def harmonic_1D(x, n, omega):
    mass=1
    hbar=1
    def factorial(n):
        if (n==0):
            return 1
        elif (n==1):
            return 1
        else:
            return n * factorial(n-1)
       
    def hp(n,x):
        if (n==0):
            return 1
        elif (n==1):
            return 2*x
        else:
            hp1 = 2*x*hp(n-1,x)
            hp2 = 2*(n-1)*hp(n-2,x)
            return hp1-hp2 
            
    harmo = np.ones(len(x),dtype=complex)
    #harmo *= (1/np.sqrt(2**n * factorial(n))) * (mass * omega/(np.pi * hbar))**0.25 
    harmo *= np.exp(-(x-0)**2 / (2 * hbar))
    harmo *= hp(n,np.sqrt(1/hbar) * (x-0))
            
    return harmo

In [3]:
#define the potential matrix
class PotentialMatrix:
    def __init__(self,
                 hbar=1,
                 xmin=-5, xmax=5, ninterval=16, totaltime=400, ntstep=1024):
        # discretisation 
        self.xx = np.linspace(xmin, xmax, ninterval)
        self.dx = self.xx[1] - self.xx[0]
        self.yy = np.linspace(xmin, xmax, ninterval)   
        self.dy = self.yy[1] - self.yy[0]
        self.zz = np.linspace(xmin, xmax, ninterval)
        self.dz = self.zz[1] - self.zz[0]
        self.aa = np.linspace(xmin, xmax, ninterval)   
        self.da = self.aa[1] - self.aa[0]
        self.x, self.y, self.z, self.a = np.meshgrid(self.xx, self.yy, self.zz, self.aa)
        
        self.hbar = hbar
        self.x0 = 0
        self.n = 0 #harmonic ground state for each vibrational mode
        
        #Diagonal Terms
        #Ag tuning modes, eV unit
        self.ome6a = 0.0740
        self.ome1 = 0.1273
        self.ome9a = 0.1568
        #B1g coupling modes, eV unit
        self.ome10a = 0.0936
        
        #quadratic
        self.Veachsingle = (1/2*self.ome6a*(self.x)**2+1/2*self.ome1*(self.y)**2
                     +1/2*self.ome9a*(self.z)**2+1/2*self.ome10a*(self.a)**2)
        self.Veach = np.zeros((2,ninterval,ninterval,ninterval,ninterval),dtype=complex) 
        self.Veach [0,:,:,:,:] = self.Veachsingle
        self.Veach [1,:,:,:,:] = self.Veachsingle
        
        #delta gap, eV unit
        self.delta = 0.4617
        self.plusdel = np.full((ninterval,ninterval,ninterval,ninterval),self.delta)     
        self.minusdel = -1*self.plusdel
  
        self.Vdelta = np.zeros((2,ninterval,ninterval,ninterval,ninterval),dtype=complex) 
        self.Vdelta [0,:,:,:,:] = self.minusdel
        self.Vdelta [1,:,:,:,:] = self.plusdel  
        
        #Vintra-state coupling (over the tuning modes v6a, v1 and v9a), eV unit
        self.kaS1Q6a = -0.0964
        self.kaS1Q1 = 0.0470
        self.kaS1Q9a = 0.1594
        
        self.kaS2Q6a = 0.1194
        self.kaS2Q1 = 0.2012
        self.kaS2Q9a = 0.0484
        
        self.VintraS1 = (self.x*self.kaS1Q6a + self.y*self.kaS1Q1 
                        + self.z*self.kaS1Q9a)
        self.VintraS2 = (self.x*self.kaS2Q6a + self.y*self.kaS2Q1 
                        + self.z*self.kaS2Q9a)  
        self.Vintra = np.zeros((2,ninterval,ninterval,ninterval,ninterval),dtype=complex) 
        self.Vintra [0,:,:,:,:] = self.VintraS1
        self.Vintra [1,:,:,:,:] = self.VintraS2 
        
        self.Vhat = self.Veach+self.Vdelta+self.Vintra
        self.diagelement = self.Vhat.flatten()
        
        #Off-Diagonal Term
        self.lamda = 0.1825
        self.sidediag = self.lamda * self.a 
        self.offdiagelement=self.sidediag.flatten()
        
        #time parameters
        self.tfinal=totaltime
        self.nt=ntstep
        self.t = np.linspace(0, self.tfinal, self.nt)
        self.dt = self.t[1] - self.t[0]
        
        #matrixsize
        self.matsize = ninterval**4
        
    #Full Matrix
    def VFullmatrix(self):
        fullmatrix = diags([self.diagelement,self.offdiagelement,self.offdiagelement],[0, self.matsize, -self.matsize]).tocsc()
        expfullmatrix = expm(-1j * (fullmatrix) * self.dt/2)
        return expfullmatrix
        
    #Diagonal Part Matrix
    def Vdiagmatrix(self):
        diagmatrix = diags([self.diagelement],[0]).tocsc()
        expdiagmatrix = expm(-1j * (diagmatrix) * self.dt/2)
        return expdiagmatrix
        
    #Off-Diagonal Part Matrix
    def Vcmatrix(self):
        offdiagmatrix = diags([self.offdiagelement,self.offdiagelement],[self.matsize, -self.matsize]).tocsc()
        expoffdiagmatrix = expm(-1j * (offdiagmatrix) * self.dt/2)
        return expoffdiagmatrix

In [4]:
Voperator = PotentialMatrix()

In [5]:
Vdiagonalpotential = Voperator.Vdiagmatrix()
Vcouplingpotential = Voperator.Vcmatrix()
fullpotential = Vdiagonalpotential@Vcouplingpotential

In [6]:
#These intermediate data can be saved for repeated use:
#scipy.sparse.save_npz('Vdiagonal Potential Matrix.npz', Vdiagonalpotential) 
#scipy.sparse.save_npz('Vcoupling Potential Matrix.npz', Vcouplingpotential) 
#scipy.sparse.save_npz('Full Potential Matrix.npz', fullpotential)

In [7]:
#The saved data can be reloaded as:
#Vdiagonalpotential = scipy.sparse.load_npz("Vdiagonal Potential Matrix.npz")
#Vcouplingpotential = scipy.sparse.load_npz("Vcoupling Potential Matrix.npz")
#fullpotential = scipy.sparse.load_npz("Full Potential Matrix.npz")

In [8]:
class Pyrazine:
    def __init__(self,
                 hbar=1,
                 xmin=-5, xmax=5, ninterval=16):
        # discretisation of coordinates
        self.xx = np.linspace(xmin, xmax, ninterval)
        self.dx = self.xx[1] - self.xx[0]
        self.yy = np.linspace(xmin, xmax, ninterval)   
        self.dy = self.yy[1] - self.yy[0]
        self.zz = np.linspace(xmin, xmax, ninterval)
        self.dz = self.zz[1] - self.zz[0]
        self.aa = np.linspace(xmin, xmax, ninterval)   
        self.da = self.aa[1] - self.aa[0]
        self.x, self.y, self.z, self.a = np.meshgrid(self.xx, self.yy, self.zz, self.aa)
        
        self.ninterval = ninterval
        self.totalelement = 2 * ninterval**4
        self.hbar = hbar
        self.n = 0 #harmonic ground state for each vibrational mode
        
        #Ag tuning modes, eV unit
        self.ome6a = 0.0740
        self.ome1 = 0.1273
        self.ome9a = 0.1568
        #B1g coupling modes, eV unit
        self.ome10a = 0.0936

        res = ninterval 
        self.dkx = 2 * np.pi / (res * self.dx)
        self.kx = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dkx
        self.dky = 2 * np.pi / (res * self.dy)
        self.ky = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dky
        self.dkz = 2 * np.pi / (res * self.dz)
        self.kz = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dkz
        self.dka = 2 * np.pi / (res * self.da)
        self.ka = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dka
        self.KX, self.KY, self.KZ, self.KA = np.meshgrid(self.kx, self.ky, self.kz, self.ka)
        
        #kinetic 
        self.oriK = (self.KX**2 * (self.ome6a/2) + self.KY**2 * (self.ome1/2) 
                     + self.KZ**2 * (self.ome9a/2) + self.KA**2 * (self.ome10a/2)) 
        self.Khat = np.zeros((2,ninterval,ninterval,ninterval,ninterval),dtype=complex) 
        self.Khat[0,:,:,:,:] = self.oriK
        self.Khat[1,:,:,:,:] = self.oriK  
        
        #quadratic potential
        self.Veachsingle = (1/2*self.ome6a*(self.x)**2+1/2*self.ome1*(self.y)**2
                     +1/2*self.ome9a*(self.z)**2+1/2*self.ome10a*(self.a)**2)
        self.Veach = np.zeros((2,ninterval,ninterval,ninterval,ninterval),dtype=complex) 
        self.Veach [0,:,:,:,:] = self.Veachsingle
        self.Veach [1,:,:,:,:] = self.Veachsingle
        
        #initial wavefunction
        self.harmo_6a_1 = np.tensordot(harmonic_1D(self.xx, self.n, self.ome6a),harmonic_1D(self.yy, self.n, self.ome1),axes=0)
        self.harmo_9a_10a = np.tensordot(harmonic_1D(self.zz, self.n, self.ome9a),harmonic_1D(self.aa, self.n, self.ome10a),axes=0)
        self.harmo_all = np.tensordot(self.harmo_6a_1,self.harmo_9a_10a,axes=0) 
        
        self.psi = np.zeros((2,ninterval,ninterval,ninterval,ninterval),dtype=complex) 
        
        #S1:S2=0:1
        self.psi [0,:,:,:,:] = np.zeros((ninterval,ninterval,ninterval,ninterval),dtype=complex)
        self.psi [1,:,:,:,:] = self.harmo_all        
        
        initnorm = np.sum(np.conjugate(self.psi)*self.psi*self.dx*self.dy*self.dz*self.da)
        self.psi *= 1/np.sqrt(initnorm)         
        self.waveint = self.psi
        
        self.psi_k = np.fft.fftn(self.waveint) 
        knorm = np.sum(np.conjugate(self.psi_k)*self.psi_k* self.dkx*self.dky*self.dkz*self.dka)
        self.psi_k *= 1/np.sqrt(knorm)
        
        #Zero Point Energy
        self.energypure = (np.sum(np.conjugate(self.waveint) * self.Veach *self.waveint * self.dx*self.dy*self.dz*self.da) + np.sum(np.conjugate(self.psi_k) * self.Khat *self.psi_k * self.dkx*self.dky*self.dkz*self.dka))              
        
        self.history = {}
    
    def TimeEvolution(self, tfinal=400, nt=1024):
        t = np.linspace(0, tfinal, nt)
        dt = t[1] - t[0]

        Kt = np.exp(-1j * (1/1) * self.Khat * dt / self.hbar)

        psi_list = []
        nor_list=[]
        
        t_list=[]
        auto_list=[]
        oneprob_list=[]
        twoprob_list=[]
        
        #record the wavefunction
        psi = np.copy(self.waveint)
        psi_list.append(psi)
        nor_list.append(np.sum(np.conjugate(psi)*psi*self.dx*self.dy*self.dz*self.da))
        
        #record the evolved time and autocorrelation
        t_list.append(t[0])
        auto_list.append(np.sum(np.conjugate(self.waveint) * psi * self.dx*self.dy*self.dz*self.da)) 
        
        #record the probabilities of S1 and S2 states
        oneprob_list.append(np.sum(np.conjugate(psi[0,:,:,:,:])*psi[0,:,:,:,:])*self.dx*self.dy*self.dz*self.da)
        twoprob_list.append(np.sum(np.conjugate(psi[1,:,:,:,:])*psi[1,:,:,:,:])*self.dx*self.dy*self.dz*self.da)
        
        
        for i in range(nt):
            
            flatpsi = psi.flatten()
            sparpsi = scipy.sparse.csc_matrix(flatpsi)
            sparpsicol = sparpsi.reshape(self.totalelement,1)
            onestep = fullpotential@sparpsicol  
            psi = onestep.toarray().reshape(2,self.ninterval,self.ninterval,self.ninterval,self.ninterval)
            
            psi =np.fft.fftn(psi) 
            psi *= Kt            
            psi = np.fft.ifftn(psi)

            flatpsi = psi.flatten()
            sparpsi = scipy.sparse.csc_matrix(flatpsi)
            sparpsicol = sparpsi.reshape(self.totalelement,1)
            onestep = fullpotential@sparpsicol  
            psi = onestep.toarray().reshape(2,self.ninterval,self.ninterval,self.ninterval,self.ninterval)
            
            #record the wavefunction, the norm, the evolved time and autocorrelation
            psi_list.append(psi)
            nor_list.append(np.sum(np.conjugate(psi)*psi*self.dx*self.dy*self.dz*self.da))             
            t_list.append(t[i])
            auto_list.append(np.sum(np.conjugate(self.waveint) * psi * self.dx*self.dy*self.dz*self.da)) 
            
            #record the probabilities of S1 and S2 states
            oneprob_list.append(np.sum(np.conjugate(psi[0,:,:,:,:])*psi[0,:,:,:,:])*self.dx*self.dy*self.dz*self.da)
            twoprob_list.append(np.sum(np.conjugate(psi[1,:,:,:,:])*psi[1,:,:,:,:])*self.dx*self.dy*self.dz*self.da) 

                        
        return t, t_list, psi_list, nor_list, auto_list, oneprob_list, twoprob_list 

In [9]:
pyrz = Pyrazine()

In [10]:
#Zero Point Energy
ZPE = pyrz.energypure

In [11]:
#Time-Dependent Properties
t, t_list, psi_list, nor_list, auto_list, oneprob_list, twoprob_list = pyrz.TimeEvolution()

In [12]:
#save the generated data
np.save('pyrazine zero point energy.npy', ZPE)
np.savetxt('pyrazine t_list.csv',t_list, delimiter=',')
np.savez('pyrazine psi_list.npz',psi_list)
np.savetxt('pyrazine nor_list.csv',nor_list, delimiter=',')
np.savetxt('pyrazine auto_list.csv',auto_list, delimiter=',')
np.savetxt('pyrazine oneprob_list.csv',oneprob_list, delimiter=',')
np.savetxt('pyrazine twoprob_list.csv',twoprob_list, delimiter=',')